## 고급-1

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from copy import copy

### event 테이블

In [3]:
df = pd.read_csv('N:/개인/데이터 분석/내배캠/4. 실전 프로젝트/events.csv')

In [4]:
df.head(5)

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
0,2198523,NaN,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel
1,1773216,NaN,3,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00,85.114.141.79,Santa Isabel,São Paulo,07500-000,Safari,Adwords,/cancel,cancel
2,2380515,NaN,3,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00,169.250.255.132,Mairiporã,São Paulo,07600-000,IE,Adwords,/cancel,cancel
3,2250597,NaN,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel
4,1834446,NaN,3,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00,161.114.4.174,São Paulo,São Paulo,09581-680,Chrome,Email,/cancel,cancel


In [6]:
# product_id 추출
event = df[df['uri'].str.contains('product/')]
event_2 = event.uri.str.split('/')
event_2 = event_2.str.get(2)

In [7]:
event['uri_num'] = event_2

C:\Users\jeong hee lee\AppData\Local\Temp\ipykernel_4556\743761526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event['uri_num'] = event_2


In [8]:
event_df = event[['id', 'user_id', 'uri_num', 'created_at']]

In [9]:
# user_id와 product_id를 기준으로 최소 created_at 구하기
event_fin = pd.DataFrame(event_df.groupby(['user_id', 'uri_num'])['created_at'].min()).reset_index()

In [10]:
# data type 변경
event_fin['created_at'] = pd.to_datetime(event_fin['created_at'])
event_fin['uri_num'] = pd.to_numeric(event_fin['uri_num'])

### order_item 테이블

In [11]:
order = pd.read_csv('N:/개인/데이터 분석/내배캠/4. 실전 프로젝트/order_items.csv')

In [12]:
order2 = order[['id', 'user_id', 'product_id', 'created_at']]

In [ ]:
# data type 변경
order2['created_at'] = pd.to_datetime(order2['created_at'])

In [14]:
# user_id, product_id 기준 최소 created_at 구하기
order_fin = pd.DataFrame(order2.groupby(['user_id', 'product_id'])['created_at'].min()).reset_index()

In [15]:
# event, order_item table merge
fin_data = pd.merge(event_fin, order_fin, how='outer', left_on=['user_id','uri_num'], right_on = ['user_id', 'product_id'])

In [16]:
fin_data.head(3)

,user_id,uri_num,created_at_x,product_id,created_at_y
0,1.0,2953,2022-07-18 10:18:43+00:00,2953,2022-07-20 10:32:05+00:00
1,1.0,4731,2022-07-18 10:54:11+00:00,4731,2022-07-20 11:05:38+00:00
2,1.0,7656,2022-07-18 11:19:54+00:00,7656,2022-07-19 11:29:28+00:00


In [17]:
# 컬럼 정리
fin_data.drop('uri_num', axis=1, inplace=True)
fin_data.rename(columns = {'created_at_x' : 'event_created_at', 'created_at_y' : 'order_created_at'}, inplace=True)
fin_data = fin_data[['user_id', 'product_id', 'event_created_at', 'order_created_at']]

In [18]:
fin_data.head(5)

,user_id,product_id,event_created_at,order_created_at
0,1.0,2953,2022-07-18 10:18:43+00:00,2022-07-20 10:32:05+00:00
1,1.0,4731,2022-07-18 10:54:11+00:00,2022-07-20 11:05:38+00:00
2,1.0,7656,2022-07-18 11:19:54+00:00,2022-07-19 11:29:28+00:00
3,2.0,25774,2022-02-20 10:25:44+00:00,2022-02-20 10:28:57+00:00
4,3.0,18177,2023-03-10 07:10:33+00:00,2023-03-10 07:14:45+00:00


In [19]:
# event, order 테이블 시간 비교
fin_data[fin_data['event_created_at'] > fin_data['order_created_at']]

,user_id,product_id,event_created_at,order_created_at


In [20]:
# boolean 컬럼 추가
boolean = []
for i in range(len(fin_data['user_id'])):
    if fin_data['event_created_at'][i] > fin_data['order_created_at'][i]:
        boolean.append('False')
    else:
        boolean.append('True')

fin_data['boolean'] = boolean

In [21]:
fin_data.tail(20)

,user_id,product_id,event_created_at,order_created_at,boolean
181727,99986.0,354,2023-09-13 23:38:19+00:00,2023-09-13 23:42:27+00:00,True
181728,99988.0,22813,2023-04-01 01:21:10+00:00,2023-04-01 01:23:51+00:00,True
181729,99989.0,15531,2023-08-28 20:44:38+00:00,2023-09-01 20:52:28+00:00,True
181730,99989.0,165,2023-08-28 23:05:51+00:00,2023-08-28 23:09:46+00:00,True
181731,99990.0,23832,2020-08-30 05:23:41+00:00,2020-08-30 05:26:30+00:00,True
181732,99992.0,11307,2022-08-06 12:01:52+00:00,2022-08-08 12:09:54+00:00,True
181733,99992.0,11362,2022-08-06 12:18:06+00:00,2022-08-09 12:30:51+00:00,True
181734,99992.0,7101,2022-08-06 14:09:43+00:00,2022-08-08 14:24:09+00:00,True
181735,99993.0,9826,2024-01-03 02:14:22+00:00,2024-01-03 02:18:57+00:00,True
181736,99996.0,18161,2023-03-13 01:59:29+00:00,2023-03-15 02:05:44+00:00,True


In [25]:
fin_data.to_csv('N:/개인/데이터 분석/내배캠/4. 실전 프로젝트/fin_data.csv', index=False)